In [ ]:
import numpy as np
import pandas as pd

import math
import numpy as np
import pandas as pd
import warnings

import matplotlib.pyplot as plt
import seaborn as sns

import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold, cross_val_score, train_test_split

import tensorflow_probability as tfp
from keras.models import Sequential
from keras.layers import Dense, LSTM

from tensorflow.keras.layers import StringLookup
import joblib
from tqdm import tqdm

#ignore warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [ ]:
# brewer2mpl.get_map args: set name set type number of colors
import matplotlib as mpl

#mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

colors = ['#1E90FF', '#f12c60', '#fff21e', '#1efff2', '#1eafff', 'blueviolet', 'sandybrown', 'magenta', 'slategray']

### Distance matrix for QPC and DPC

### QPC

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

tfd = tfp.distributions

In [ ]:
def qloss(y_true, y_pred, n_q=99):
    q = np.array(range(1, n_q + 1))
    left = (q / (n_q + 1) - 1) * (y_true - y_pred)
    right = q / (n_q + 1) * (y_true - y_pred)
    return K.mean(K.maximum(left, right))

from keras.callbacks import *

def QRNN_2(num_quantiles):
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(64, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(32, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  outputs=Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

def QRNN_3(num_quantiles):
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  outputs=Dense(num_quantiles)(mean_x)
  model = keras.Model(inputs=mean_inputs, outputs=outputs)
  return model

In [ ]:
from tqdm import tqdm

# use the structure of QRNN_3
distance_matrix = np.zeros((77, 23))

for i in tqdm(range(77)):
  nrm0 = nrm[i]
  speed0 = nrm0[:, 0::3].flatten()
    
  power_list = []

  speed_list = np.linspace(np.percentile(speed0, 0.5), np.percentile(speed0, 99.5), 20)
  speed_list = speed_list.reshape(len(speed_list), 1)

  for j in range(23):
    num_quantiles = 9
    model = QRNN_3(num_quantiles)
    model.load_weights("Nw100/model_quantile/model_%s_%s.h5"%(i,j))
    power_list.append(model.predict(speed_list))

  power_list = np.array(power_list)
  power_list[power_list>1.1] = 1.1
  power_list[power_list<0] = 0

  for k in range(23):
    tmp = np.delete(power_list, (k), axis = 0)
    distance_all = [(np.mean((tmp[i] - power_list[k])**2))**0.5 for i in range(len(tmp))]
    sort_array = np.argsort(distance_all)[:-3]
    filter_distance = [distance_all[i] for i in sort_array]
    distance_matrix[i][k] = np.mean(filter_distance)

Profile monitoring

In [ ]:
import scipy

def calculate_cdf(dis, model, alpha):
  value_list = np.linspace(min(dis)-max(dis), 2*max(dis), 2000)

  for i in range(len(value_list)-1):
      value1 = model.integrate_box_1d(low = -np.Inf, high = value_list[i])
      value2 = model.integrate_box_1d(low = -np.Inf, high = value_list[i+1])
      if value1 <= alpha and value2 >= alpha:
          return (value_list[i] + value_list[i+1])/2

  return np.nan

In [ ]:
np.save('Nw100/distance4qpc.npy', distance_matrix)

In [ ]:
distance_matrix = np.load('Nw100/distance4qpc.npy',)

In [ ]:
np.max(distance_matrix)

0.6682915348324133

In [ ]:
df = pd.DataFrame(data=distance_matrix.T)
df.to_csv('Nw100/distance4qpc.csv', index=False)

In [ ]:
degree_matrix = np.zeros((77, 23))

for i in tqdm(range(77)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/100  # 500 is the length of the data segment

degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1

100%|██████████| 77/77 [00:00<00:00, 8551.42it/s]


In [ ]:
sns.heatmap(degree_matrix)

In [ ]:
distance_matrix = np.load('Nw100/distance4dpc.npy',)

In [ ]:
sns.heatmap(distance_matrix)

KDE

In [ ]:
# KDE
from tqdm import tqdm

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)

UCL_list = []
for i in tqdm(range(77)):
  kde_model = scipy.stats.gaussian_kde(distance_matrix[i], bw_method = 0.2)
  cdf_list = [calculate_cdf(distance_matrix[i], kde_model, confidence) for confidence in np.linspace(0.01,0.99,99)]
  UCL_list.append(cdf_list[ind])

100%|██████████| 77/77 [05:23<00:00,  4.20s/it]


In [ ]:
predict_matrix = np.zeros((77, 23))

for i in range(77):
  for j in range(23):
    if distance_matrix[i][j] > UCL_list[i]:
      predict_matrix[i][j] = 1
    else:
      pass

In [ ]:
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

In [ ]:
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass
    
precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

recall 0.6129032258064516
precision 0.2261904761904762
f1-score 0.33043478260869563


In [ ]:
FP

74

BCKDE

In [ ]:
predict_matrix = pd.read_csv('Nw100/bckde_qpc_0.8_new.csv').to_numpy()

In [ ]:
degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

In [ ]:
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass
    
precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

recall 0.1935483870967742
precision 0.3157894736842105
f1-score 0.24000000000000002


### DPC

In [ ]:
import os
import numpy as np
import pandas as pd
import time
import math
import tensorflow as tf
from keras import Model
from keras.models import Sequential
import keras.backend as K
from keras.layers import *
from keras.models import load_model
from keras.utils.vis_utils import plot_model

from keras import regularizers
from keras.constraints import non_neg
from keras.initializers import RandomUniform,RandomNormal
from matplotlib import pyplot as plt

from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold, cross_val_score, train_test_split

tfd = tfp.distributions

In [ ]:
def BNN_normal():
  mean_inputs = tf.keras.Input((1,1))
  mean_x = LSTM(32, return_sequences=True,)(mean_inputs)
  mean_x=LSTM(16, return_sequences=False)(mean_x)
  mean_x = Dense(16,)(mean_x)
  mean_x = Dense(8,)(mean_x)

  mean_x = Dense(2)(mean_x)
  outputs=tfp.layers.DistributionLambda(lambda t: tfd.Normal(loc=t[..., 0],scale=1e-3+tf.math.softplus(0.05 * t[..., 1])))(mean_x)
  # outputs = Dense(1)(mean_x)
  model = keras.Model(inputs=[mean_inputs], outputs=[outputs])
  return model

import keras.backend as K
import math as m

def negative_loglikelihood(targets,estimated_distribution):
  return -estimated_distribution.log_prob(targets)

In [ ]:
from scipy.stats import norm
import scipy
import numpy as np
from numba import jit,njit
from tqdm import *

def calculate_kl_divergence(dis1, dis2):
    kl_div = 0
    min_1 = dis1[0] - 2*dis1[1]
    min_2 = dis2[0] - 2*dis2[1]
    max_1 = dis1[0] + 2*dis1[1]
    max_2 = dis2[0] + 2*dis2[1]
    min_value = min(min_1, min_2)
    max_value = max(max_1, max_2)
    values = np.linspace(min_value, max_value, 20)
    
    for x in values:
        px = norm.pdf(x, loc = dis1[0], scale = dis1[1])
        qx = norm.pdf(x, loc = dis2[0], scale = dis2[1])
        if px < 1e-16:
            px = 1e-16
            
        if qx < 1e-16:
            qx = 1e-16
        
        kl_div += px*np.log(px/qx)

    return kl_div

In [ ]:
# For QiPao Plot
distance_matrix = np.zeros((77, 23)) # 46 for nw500; 23 for nw4000

for i in tqdm(range(77)):
  nrm0 = nrm[i]
  speed0 = nrm0[:, 0::3].flatten()

  power_list = []

  speed_list = np.linspace(np.percentile(speed0, 0.5), np.percentile(speed0, 99.5), 20)
  speed_list = speed_list.reshape(len(speed_list), 1)

  for j in range(23):
    num_quantiles = 9
    model = BNN_normal()
    model.load_weights("Nw100/model_density/model_%s_%s.h5"%(i,j))
    power_list.append(model(speed_list))


  tmp_distance = np.zeros((23, 23))
  for m in range(23):
    mean_list1 = power_list[m].mean().numpy().tolist()
    std_list1 = power_list[m].stddev().numpy().tolist()
    length = len(mean_list1)
    for j in range(m+1, 23):
      mean_list2 = power_list[j].mean().numpy().tolist()
      std_list2 = power_list[j].stddev().numpy().tolist()
      tmp_distance[m][j] = np.mean([calculate_kl_divergence([mean_list1[k],std_list1[k]], [mean_list2[k],std_list2[k]]) for k in range(length)])
      tmp_distance[j][m] = tmp_distance[m][j]


  for k in range(23):
    distance_all = np.delete(tmp_distance[k], k)
    sort_array = np.argsort(distance_all)[:-3]
    filter_distance = [distance_all[i] for i in sort_array]
    distance_matrix[i][k] = np.mean(filter_distance)

100%|██████████| 77/77 [1:12:07<00:00, 56.21s/it]


In [ ]:
np.save('Nw100/distance4dpc.npy', distance_matrix)

In [ ]:
distance_matrix = np.load('Nw100/distance4dpc.npy',)

In [ ]:
df = pd.DataFrame(data=distance_matrix.T)
df.to_csv('Nw100/distance4dpc.csv', index=False)

KDE

In [ ]:
import scipy

def calculate_cdf(dis, model, alpha):
  value_list = np.linspace(min(dis)-max(dis), 2*max(dis), 2000)

  for i in range(len(value_list)-1):
      value1 = model.integrate_box_1d(low = -np.Inf, high = value_list[i])
      value2 = model.integrate_box_1d(low = -np.Inf, high = value_list[i+1])
      if value1 <= alpha and value2 >= alpha:
          return (value_list[i] + value_list[i+1])/2

  return np.nan

In [ ]:
# KDE
from tqdm import tqdm

ind = np.linspace(0.01,0.99,99).searchsorted(0.95)

UCL_list = []
for i in tqdm(range(77)):
  kde_model = scipy.stats.gaussian_kde(distance_matrix[i], bw_method = 0.8)
  cdf_list = [calculate_cdf(distance_matrix[i], kde_model, confidence) for confidence in np.linspace(0.01,0.99,99)]
  UCL_list.append(cdf_list[ind])

100%|██████████| 77/77 [05:37<00:00,  4.39s/it]


In [ ]:
degree_matrix = np.zeros((77, 23))

for i in tqdm(range(77)):
  nrm0 = nrm[i]
  label0 = nrm0[:, 2::3]
  degree_matrix[i] = np.sum(label0, axis=0)/100  # 500 is the length of the data segment

100%|██████████| 77/77 [00:00<00:00, 3918.06it/s]


In [ ]:
predict_matrix = np.zeros((77, 23))

for i in range(77):
  for j in range(23):
    if distance_matrix[i][j] > UCL_list[i]:
      predict_matrix[i][j] = 1
    else:
      pass


degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1
pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

# calculate precision and recall
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass
    
precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

BCKDE

In [ ]:
predict_matrix = pd.read_csv('Nw100/bckde_dpc_0.8_new.csv').to_numpy()

In [ ]:
np.sum(degree_matrix)

31.0

In [ ]:
degree_matrix[degree_matrix < 0.6] = 0
degree_matrix[degree_matrix >= 0.6] = 1

pred = predict_matrix.flatten()
raw = degree_matrix.flatten()

# calculate precision and recall
TP, FP, FN = 0, 0, 0

for i in range(len(pred)):
    if pred[i] == 1 and raw[i] == 1:
        TP += 1
    elif pred[i] == 1 and raw[i] == 0:
        FP += 1
    elif pred[i] == 0 and raw[i] == 1:
        FN += 1
    else:
        pass
    
precision = TP/(TP + FP)
recall = TP/(TP + FN)

print('recall', recall)
print('precision', precision)
print('f1-score', (2*precision*recall)/(precision+recall))

recall 0.3548387096774194
precision 0.046610169491525424
f1-score 0.08239700374531836
